In [1]:
import numpy as np
import pandas as pd


def get_precip_data():
    return pd.read_csv('C:/Users/ericz/Desktop/CMPT353/e1/precipitation.csv', parse_dates=[2])


def date_to_month(d):
    # You may need to modify this function, depending on your data types.
    return d.dt.strftime('%Y-%m')


def pivot_months_pandas(data):
    """
    Create monthly precipitation totals for each station in the data set.
    
    This should use Pandas methods to manipulate the data.
    """
    data.insert(loc=3,column='month',value = date_to_month(data['date']))
    group = data.groupby(['name','month'])
    sum_pre = group.aggregate({'precipitation': 'sum'}).reset_index()
    totals = sum_pre.pivot(index = 'name',columns = 'month',values = 'precipitation')
    sum_counts = group.aggregate('count').reset_index()
    counts = sum_counts.pivot(index = 'name',columns = 'month',values = 'elevation')
    # ...
    return totals, counts


def pivot_months_loops(data):
    """
    Create monthly precipitation totals for each station in the data set.
    
    This does it the hard way: using Pandas as a dumb data store, and iterating in Python.
    """
    # Find all stations and months in the data set.
    stations = set()
    months = set()
    for i,r in data.iterrows():
        stations.add(r['name'])
        m = '%04i-%02i' % (r['date'].year, r['date'].month)
        months.add(m)

    # Aggregate into dictionaries so we can look up later.
    stations = sorted(list(stations))
    row_to_station = dict(enumerate(stations))
    station_to_row = {s: i for i,s in row_to_station.items()}
    
    months = sorted(list(months))
    col_to_month = dict(enumerate(months))
    month_to_col = {m: i for i,m in col_to_month.items()}

    # Create arrays for the data, and fill them.
    precip_total = np.zeros((len(row_to_station), 12), dtype=np.uint)
    obs_count = np.zeros((len(row_to_station), 12), dtype=np.uint)

    for _, row in data.iterrows():
        m = '%04i-%02i' % (row['date'].year, row['date'].month)
        r = station_to_row[row['name']]
        c = month_to_col[m]

        precip_total[r, c] += row['precipitation']
        obs_count[r, c] += 1

    # Build the DataFrames we needed all along (tidying up the index names while we're at it).
    totals = pd.DataFrame(
        data=precip_total,
        index=stations,
        columns=months,
    )
    totals.index.name = 'name'
    totals.columns.name = 'month'
    
    counts = pd.DataFrame(
        data=obs_count,
        index=stations,
        columns=months,
    )
    counts.index.name = 'name'
    counts.columns.name = 'month'
    
    return totals, counts


def main():
    data = get_precip_data()
    totals, counts = pivot_months_loops(data)
    totals.to_csv('totals.csv')
    counts.to_csv('counts.csv')
    np.savez('monthdata.npz', totals=totals.values, counts=counts.values)
    


if __name__ == '__main__':
    main()


          station                    name       date  latitude  longitude  \
0     CA001101158  BURNABY SIMON FRASER U 2016-01-01   49.2833  -122.9167   
1     CA001101158  BURNABY SIMON FRASER U 2016-01-02   49.2833  -122.9167   
2     CA001101158  BURNABY SIMON FRASER U 2016-01-03   49.2833  -122.9167   
3     CA001101158  BURNABY SIMON FRASER U 2016-01-04   49.2833  -122.9167   
4     CA001101158  BURNABY SIMON FRASER U 2016-01-05   49.2833  -122.9167   
...           ...                     ...        ...       ...        ...   
3167  CA002204101           YELLOWKNIFE A 2016-12-27   62.4667  -114.4333   
3168  CA002204101           YELLOWKNIFE A 2016-12-28   62.4667  -114.4333   
3169  CA002204101           YELLOWKNIFE A 2016-12-29   62.4667  -114.4333   
3170  CA002204101           YELLOWKNIFE A 2016-12-30   62.4667  -114.4333   
3171  CA002204101           YELLOWKNIFE A 2016-12-31   62.4667  -114.4333   

      elevation  precipitation  
0         366.0              0  
1        

In [2]:
data = get_precip_data()

In [3]:
totals_pd, counts_pd = pivot_months_pandas(data)

In [4]:
totals_loop, counts_loop = pivot_months_loops(data)

In [5]:
totals_loop  # look how nicely Jupyter displays a DataFrame...

month,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08,2016-09,2016-10,2016-11,2016-12
name,,,,,,,,,,,,
BURNABY SIMON FRASER U,1809,1482,2159,374,606,428,442,68,956,1429,1349,1846
CALGARY INTL A,121,22,46,40,683,616,2061,834,253,276,27,224
GANDER INTL A,896,1232,992,1106,1236,1015,1264,1096,1026,2422,1184,1099
HALIFAX INTL A,1491,1473,1452,1595,1001,725,734,445,845,2185,1305,1835
REVELSTOKE,930,561,533,204,546,748,662,274,785,1620,1293,295
SHERBROOKE,183,982,621,471,667,671,525,1599,433,1176,594,627
TORONTO LESTER B. PEARSON INT',363,491,729,681,346,269,399,667,663,452,497,821
VANCOUVER INTL A,1682,1304,1616,242,516,582,328,138,784,2034,2402,1520
YELLOWKNIFE A,220,90,28,120,98,306,154,280,488,158,176,116


In [6]:
abs(totals_pd.values - totals_loop.values).sum() # the two totals should be the same

0

In [7]:
abs(counts_pd.values - counts_loop.values).sum() # ...counts too.

0

In [8]:
%timeit pivot_months_pandas(data)

ValueError: cannot insert month, already exists

In [9]:
%timeit pivot_months_loops(data)

1.11 s ± 25.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
